In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# openai work
from openai import OpenAI

client = OpenAI()
def stream_openai(client, role, message):
    stream = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": role, "content": message}],
        stream=True,
    )
    message = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            message += chunk.choices[0].delta.content
    return message

# m = stream_openai(client, "user", "hello, how are you?")
# m

In [3]:
# Preprocessor test
from pathlib import Path

from ragna.source_storages import RagnaDemoSourceStorage
from ragna.core import (
    LocalDocument, Document
)


storage = RagnaDemoSourceStorage()

# md_files = 
# ragna_documents = [
#             (
#                 document
#                 if isinstance(document, Document)
#                 else LocalDocument.from_path(document)
#             )
#             for document in md_files
#         ]
# storage.store("ragna_docs", ragna_documents)

temporal_doc_paths = [p for p in Path.cwd().joinpath("temporal_docs").iterdir() if p.suffix == ".md"]
temporal_docs = [
    LocalDocument.from_path(path) for path in temporal_doc_paths
]
storage.store("temporal_docs", temporal_docs)

## Run ragna with no preprocessing on temporal_docs

In [4]:
from typing import Optional

from ragna.assistants import RagnaDemoAssistant

from ragna import Rag
from ragna.assistants import Gpt4


chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="temporal_docs",
)
_ = await chat.prepare()
print(await chat.answer("What happened last year?"))

Last year, in 2023, a rocket was built.


## Run with Preprocessor

In [5]:
from typing import Optional
from ragna.core import (
QueryPreprocessor, MetadataFilter, ProcessedQuery
)

base_prompt = (
"""You are a llm agent that is responsible for rewriting an input prompt for a RAG application. 
A question might contain hidden context that may not be recognized by the embedding model. 
An embedding of a question might not generate a close match to an embedding of a statement
that contains the answer to the question. Although this is technically something that should
be solved on the embedding model side, it is usually solved by rephrasing the question
before using it to retrieve sources. For example a question like "What happened last month?" 
likely won't get any close matches. Rephrasing the prompt to "What happened in December 2021?"
given that the question is asked January 2022. Things that may be important to consider when 
reworking the prompt would be the current context, that may that is not explicitly asked in the 
question but can be inferred, for instance the current date. 

current date: 2025-01-24


Please reword the following prompt 
and only return the reworded prompt. I do not need to know your reasoning:


""")

class TestPreprocessor(QueryPreprocessor):

    def __init__(self):
        # self.storage=storage
        # self.assistant=assistant
        self.messages = []

    def ask_assistant(self, prompt):
        instruction = (base_prompt + prompt)
        print(instruction)
        assistant_answer = stream_openai(client, "user", instruction)
        return assistant_answer

    def process(self, query: str, metadata_filter: Optional[MetadataFilter]):
        processed_query = self.ask_assistant(query)
        return ProcessedQuery(
            original_query=query,
            processed_query=processed_query,
            metadata_filter=None,
            processor_name=self.display_name()
        )

In [6]:
preprocessor = TestPreprocessor()
processed_query = preprocessor.process("What happened last year?", None)

You are a llm agent that is responsible for rewriting an input prompt for a RAG application. 
A question might contain hidden context that may not be recognized by the embedding model. 
An embedding of a question might not generate a close match to an embedding of a statement
that contains the answer to the question. Although this is technically something that should
be solved on the embedding model side, it is usually solved by rephrasing the question
before using it to retrieve sources. For example a question like "What happened last month?" 
likely won't get any close matches. Rephrasing the prompt to "What happened in December 2021?"
given that the question is asked January 2022. Things that may be important to consider when 
reworking the prompt would be the current context, that may that is not explicitly asked in the 
question but can be inferred, for instance the current date. 

current date: 2025-01-24


Please reword the following prompt 
and only return the reworded prompt. 

In [7]:
print(processed_query.processed_query)

What happened in 2024?


In [8]:
chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="temporal_docs",
)
_ = await chat.prepare()
print(await chat.answer(processed_query.processed_query))

In 2024, they went to the moon.
